In [5]:
import os
import glob
import random
import math
import json
import re
from pathlib import Path
import pandas as pd
from sklearn.model_selection import GroupShuffleSplit
import torch
from torch.utils.data import Dataset, DataLoader, WeightedRandomSampler
from PIL import Image
import torchvision.transforms as T
import timm
import torch.nn as nn
from torch.optim import AdamW
from torch.optim.lr_scheduler import OneCycleLR
from torchmetrics.classification import MulticlassAccuracy
from tqdm.auto import tqdm
import torch.nn.functional as F
from sklearn.metrics import classification_report, confusion_matrix
import numpy as np

In [ ]:
DATA_ROOT = Path("/kaggle/input/behaviours-features-merged/Behaviors_Features_Final")

# Collect all images recursively and derive labels from behavior folder name.
records = []
for behavior_dir in sorted([p for p in DATA_ROOT.iterdir() if p.is_dir()]):
    behavior = behavior_dir.name  # e.g., 'Looking_Forward'
    for id_dir in behavior_dir.glob("*"):
        if not id_dir.is_dir(): 
            continue
        for seq_dir in id_dir.glob("*"):
            if not seq_dir.is_dir():
                continue
            # Group key: person+sequence folder to avoid near-duplicate leakage
            group_key = f"{behavior}/{id_dir.name}/{seq_dir.name}"
            for img_path in seq_dir.rglob("*.png"):
                records.append({
                    "path": str(img_path),
                    "label": behavior,
                    "group": group_key,
                    "person": id_dir.name,
                    "sequence": seq_dir.name,
                })

df = pd.DataFrame(records)

# Remove 'Standing' label from the dataset
df = df[df["label"] != "Standing"].reset_index(drop=True)

print("Total images:", len(df))
df.head()

Total images: 632620


,path,label,group,person,sequence
0,/kaggle/input/behaviours-features-merged/Behav...,Looking_Forward,Looking_Forward/ID6/Forward44_id6_Act1_rgb,ID6,Forward44_id6_Act1_rgb
1,/kaggle/input/behaviours-features-merged/Behav...,Looking_Forward,Looking_Forward/ID6/Forward44_id6_Act1_rgb,ID6,Forward44_id6_Act1_rgb
2,/kaggle/input/behaviours-features-merged/Behav...,Looking_Forward,Looking_Forward/ID6/Forward44_id6_Act1_rgb,ID6,Forward44_id6_Act1_rgb
3,/kaggle/input/behaviours-features-merged/Behav...,Looking_Forward,Looking_Forward/ID6/Forward44_id6_Act1_rgb,ID6,Forward44_id6_Act1_rgb
4,/kaggle/input/behaviours-features-merged/Behav...,Looking_Forward,Looking_Forward/ID6/Forward44_id6_Act1_rgb,ID6,Forward44_id6_Act1_rgb


In [ ]:
# Map class names to indices; keep a clean label list for the model head.
class_names = sorted(df["label"].unique())
class2idx = {c:i for i,c in enumerate(class_names)}
df["y"] = df["label"].map(class2idx)

# Split train/val/test based on person
from sklearn.model_selection import GroupShuffleSplit
gss = GroupShuffleSplit(n_splits=1, test_size=0.15, random_state=42)
trainval_idx, test_idx = next(gss.split(df, groups=df["person"]))
df_trainval, df_test = df.iloc[trainval_idx].reset_index(drop=True), df.iloc[test_idx].reset_index(drop=True)

gss2 = GroupShuffleSplit(n_splits=1, test_size=0.15, random_state=123)
tr_idx, va_idx = next(gss2.split(df_trainval, groups=df_trainval["person"]))
df_train, df_val = df_trainval.iloc[tr_idx].reset_index(drop=True), df_trainval.iloc[va_idx].reset_index(drop=True)

print(len(df_train), len(df_val), len(df_test))
class_names

464108 77042 91470


['Looking_Forward',
 'Raising_Hand',
 'Reading',
 'Sleeping',
 'Standing',
 'Turning_Around',
 'Writing']

In [ ]:
def _natural_key(value: str):
    return [int(tok) if tok.isdigit() else tok.lower() for tok in re.findall(r"\d+|\D+", str(value))]

def to_sequence_df(df_imgs: pd.DataFrame) -> pd.DataFrame:
    grouped = (
        df_imgs.groupby("group")
        .agg(paths=("path", list), label=("label", "first"), y=("y", "first"))
        .reset_index()
    )
    grouped["paths"] = grouped["paths"].apply(lambda items: sorted(items, key=_natural_key))
    return grouped

df_train_seq = to_sequence_df(df_train)
df_val_seq = to_sequence_df(df_val)
df_test_seq = to_sequence_df(df_test)
print(len(df_train_seq), len(df_val_seq), len(df_test_seq))

seq_counts = df_train_seq.groupby("label").size().reindex(class_names, fill_value=0)
seq_counts_clipped = seq_counts.replace(0, 1)
sequence_class_weights = (1.0 / seq_counts_clipped)
sequence_class_weights = sequence_class_weights / sequence_class_weights.sum() * len(sequence_class_weights)
sequence_class_weights

In [ ]:
IMG_SIZE = 224  # Base spatial resolution.

# image augmentation (train)
train_tfms = T.Compose([
    T.RandomResizedCrop(IMG_SIZE, scale=(0.5, 1.0)),
    T.RandomHorizontalFlip(p=0.5),
    T.RandomRotation(degrees=10),
    T.ColorJitter(brightness=0.3, contrast=0.3, saturation=0.3, hue=0.05),
    T.RandomGrayscale(p=0.1),
    T.GaussianBlur(kernel_size=3, sigma=(0.1, 2.0)),
    T.ToTensor(),
    T.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225)),
    T.RandomErasing(p=0.25, scale=(0.02, 0.2), ratio=(0.3, 3.3), value='random'),
])

valid_tfms = T.Compose([
    T.Resize(int(IMG_SIZE * 1.14)),
    T.CenterCrop(IMG_SIZE),
    T.ToTensor(),
    T.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225)),
])

class SequenceDataset(Dataset):
    """Return a fixed-length clip of frames for sequence-level classification."""
    def __init__(self, df_seq: pd.DataFrame, transforms, clip_len: int = 8, train: bool = False):
        self.paths_list = df_seq["paths"].tolist()
        self.labels = df_seq["y"].astype(int).tolist()
        self.transforms = transforms
        self.clip_len = clip_len
        self.train = train

    def __len__(self):
        return len(self.paths_list)

    def _sample_indices_train(self, n_frames: int):
        # Random continuous clip for training
        if n_frames >= self.clip_len:
            max_start = n_frames - self.clip_len
            start = random.randint(0, max_start)
            return list(range(start, start + self.clip_len))
        # Pad with the last frame if not enough
        return list(range(n_frames)) + [n_frames - 1] * (self.clip_len - n_frames)

    def _sample_indices_eval(self, n_frames: int):
        # Deterministic uniform sampling for validation/test
        if n_frames >= self.clip_len:
            return np.linspace(0, n_frames - 1, self.clip_len).astype(int).tolist()
        return list(range(n_frames)) + [n_frames - 1] * (self.clip_len - n_frames)

    def __getitem__(self, idx):
        paths = self.paths_list[idx]
        label = self.labels[idx]
        n = len(paths)
        indices = self._sample_indices_train(n) if self.train else self._sample_indices_eval(n)

        frames = []
        for frame_idx in indices:
            img = Image.open(paths[frame_idx]).convert("RGB")
            frames.append(self.transforms(img))
        clip = torch.stack(frames, dim=0)
        return clip, label

CLIP_LEN = 8
train_ds = SequenceDataset(df_train_seq, train_tfms, clip_len=CLIP_LEN, train=True)
val_ds   = SequenceDataset(df_val_seq,   valid_tfms, clip_len=CLIP_LEN, train=False)
test_ds  = SequenceDataset(df_test_seq,  valid_tfms, clip_len=CLIP_LEN, train=False)

seq_weight_lookup = sequence_class_weights.to_dict()
train_sample_weights = df_train_seq["label"].map(seq_weight_lookup).astype(float).values
train_sampler = WeightedRandomSampler(
    torch.as_tensor(train_sample_weights, dtype=torch.double),
    num_samples=len(train_sample_weights),
    replacement=True,
)

BATCH_SIZE = 16
NUM_WORKERS = 0  # Avoid notebook exceptions by setting to 0
PIN_MEMORY = False

train_dl = DataLoader(train_ds, batch_size=BATCH_SIZE, sampler=train_sampler, num_workers=NUM_WORKERS, pin_memory=PIN_MEMORY)
val_dl   = DataLoader(val_ds,   batch_size=BATCH_SIZE, shuffle=False,    num_workers=NUM_WORKERS, pin_memory=PIN_MEMORY)
test_dl  = DataLoader(test_ds,  batch_size=BATCH_SIZE, shuffle=False,    num_workers=NUM_WORKERS, pin_memory=PIN_MEMORY)
# ...existing code...

In [ ]:
SEED = 42

torch.manual_seed(SEED)
np.random.seed(SEED)
random.seed(SEED)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

N_CLASSES = len(class_names)
MODEL_NAME = "convnext_small.fb_in22k_ft_in1k"

class TemporalMeanNet(nn.Module):
    def __init__(self, backbone_name: str, n_classes: int):
        super().__init__()
        self.backbone = timm.create_model(
            backbone_name,
            pretrained=True,
            num_classes=0,
            global_pool="avg",
            drop_path_rate=0.2,
        )
        self.embed_dim = self.backbone.num_features
        # Add regularization to the head
        self.head = nn.Sequential(
            nn.LayerNorm(self.embed_dim),
            nn.Dropout(p=0.2),
            nn.Linear(self.embed_dim, n_classes),
        )

    def forward(self, clips):
        # clips: (batch, time, channels, height, width)
        b, t, c, h, w = clips.shape
        clips = clips.view(b * t, c, h, w)
        feats = self.backbone(clips)  # (b * t, feat_dim)
        feats = feats.view(b, t, -1).mean(dim=1)
        return self.head(feats)

model = TemporalMeanNet(MODEL_NAME, N_CLASSES).to(device)
use_amp = device.type == "cuda"

# Mixup/CutMix + SoftTargetCrossEntropy
from timm.data import Mixup
from timm.loss import SoftTargetCrossEntropy

mixup_fn = Mixup(
    mixup_alpha=0.2, cutmix_alpha=1.0, prob=0.5, switch_prob=0.0,
    mode='batch', label_smoothing=0.0, num_classes=N_CLASSES
)
criterion = SoftTargetCrossEntropy()

optimizer = AdamW(model.parameters(), lr=2e-4, weight_decay=5e-2)
EPOCHS = 10
steps_per_epoch = len(train_dl)
scheduler = OneCycleLR(optimizer, max_lr=2e-4, epochs=EPOCHS, steps_per_epoch=steps_per_epoch)
metric_acc = MulticlassAccuracy(num_classes=N_CLASSES).to(device)
scaler = torch.amp.GradScaler('cuda', enabled=use_amp)

def run_one_epoch(dataloader, train=True):
    model.train(train)
    total_loss = 0.0
    metric_acc.reset()
    pbar = tqdm(dataloader, leave=False)
    for clips, targets in pbar:
        clips = clips.to(device, non_blocking=True)
        targets = targets.to(device, non_blocking=True)

        if train and mixup_fn is not None:
            hard_targets = targets.clone()
        with torch.set_grad_enabled(train):
            with torch.amp.autocast('cuda', enabled=use_amp):
                if train and mixup_fn is not None:
                    clips, soft_targets = mixup_fn(clips, targets)
                    logits = model(clips)
                    loss = criterion(logits, soft_targets)
                else:
                    logits = model(clips)
                    loss = nn.functional.cross_entropy(logits, targets)

        if train:
            optimizer.zero_grad()
            scaler.scale(loss).backward()
            scaler.unscale_(optimizer)
            nn.utils.clip_grad_norm_(model.parameters(), max_norm=2.0)
            scaler.step(optimizer)
            scaler.update()
            scheduler.step()
            current_lr = scheduler.get_last_lr()[0]
            pbar.set_postfix(loss=loss.item(), lr=f"{current_lr:.2e}")
        else:
            pbar.set_postfix(loss=loss.item())

        total_loss += loss.item() * clips.size(0)
        preds = logits.argmax(dim=1)
        # Calculate accuracy with hard targets during Mixup
        metric_acc.update(preds, hard_targets if (train and mixup_fn is not None) else targets)

    avg_loss = total_loss / len(dataloader.dataset)
    avg_acc = metric_acc.compute().item()
    return avg_loss, avg_acc

best_val = 0.0
patience, bad_epochs = 3, 0  # Early stopping
for epoch in range(1, EPOCHS + 1):
    tr_loss, tr_acc = run_one_epoch(train_dl, train=True)
    va_loss, va_acc = run_one_epoch(val_dl, train=False)
    print(f"Epoch {epoch:02d} | train loss {tr_loss:.4f} acc {tr_acc:.4f} | val loss {va_loss:.4f} acc {va_acc:.4f}")
    if va_acc > best_val:
        best_val = va_acc
        bad_epochs = 0
        torch.save({
            "model_name": MODEL_NAME,
            "state_dict": model.state_dict(),
            "class_names": class_names,
            "clip_len": CLIP_LEN,
        }, "/kaggle/working/best_model_convnext_small_in22ft1k_sequence_based_group_split_augmented.pth")
        print("Saved new best model.")
    else:
        bad_epochs += 1
        if bad_epochs >= patience:
            print("Early stopping.")
            break

/usr/local/lib/python3.11/dist-packages/timm/models/_factory.py:138: UserWarning: Mapping deprecated model name convnext_small_in22ft1k to current convnext_small.fb_in22k_ft_in1k.
  model = create_fn(


model.safetensors:   0%|          | 0.00/201M [00:00<?, ?B/s]

/tmp/ipykernel_80/1084503723.py:31: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler = torch.cuda.amp.GradScaler(enabled=use_amp)


  0%|          | 0/2884 [00:00<?, ?it/s]

/tmp/ipykernel_80/1084503723.py:44: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast(enabled=use_amp):


KeyboardInterrupt: 

In [ ]:
ckpt = torch.load("/kaggle/working/best_model_convnext_small_in22ft1k_sequence_based_group_split_augmented.pth", map_location=device)
eval_model = TemporalMeanNet(ckpt["model_name"], len(ckpt["class_names"])).to(device)
eval_model.load_state_dict(ckpt["state_dict"])
eval_model.eval()
all_preds, all_targs = [], []
with torch.no_grad():
    for clips, targets in tqdm(test_dl):
        clips = clips.to(device)
        logits = eval_model(clips)
        preds = logits.argmax(1).cpu().numpy()
        all_preds.append(preds)
        all_targs.append(targets.numpy())

y_pred = np.concatenate(all_preds)
y_true = np.concatenate(all_targs)

print(classification_report(y_true, y_pred, target_names=class_names))
print(confusion_matrix(y_true, y_pred))


  0%|          | 0/566 [00:00<?, ?it/s]

                 precision    recall  f1-score   support

Looking_Forward       0.99      1.00      1.00      4333
   Raising_Hand       1.00      0.99      0.99      2979
        Reading       1.00      1.00      1.00      6975
       Sleeping       1.00      1.00      1.00      8401
       Standing       1.00      1.00      1.00      1195
 Turning_Around       1.00      1.00      1.00      5939
       Writting       1.00      1.00      1.00      6361

       accuracy                           1.00     36183
      macro avg       1.00      1.00      1.00     36183
   weighted avg       1.00      1.00      1.00     36183

[[4333    0    0    0    0    0    0]
 [  22 2952    0    5    0    0    0]
 [   0    2 6966    0    0    6    1]
 [   0    2    0 8377    0   21    1]
 [   0    1    0    4 1190    0    0]
 [   0    0    0    0    0 5939    0]
 [   0    0    0    0    0    0 6361]]
